<a href="https://colab.research.google.com/github/rasla54/-project/blob/master/BertQA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install wikipedia

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 49.0MB/s 
     |████████████████████████████████| 3.8MB 39.3MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=09438f4b0738677f674e8f43255e2389a4f9382387204710fdca30e1df24f34b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=39810a83ab93dd0888fb1b60f1c789ff87a0120cfb9515eb3f72061da88921d7
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import torch
import wikipedia

In [3]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [6]:
while True:

  question=input("Question:" )
  search = wikipedia.search(question,results=10)
  answer_text= wikipedia.summary(search[0])
  answer_text= answer_text[:2000]

  input_ids = tokenizer.encode(question, answer_text)

  # print('The input has a total of {:} tokens.'.format(len(input_ids)))
  tokens = tokenizer.convert_ids_to_tokens(input_ids)


  sep_index = input_ids.index(tokenizer.sep_token_id)
  num_seg_a = sep_index + 1
  num_seg_b = len(input_ids) - num_seg_a
  segment_ids = [0]*num_seg_a + [1]*num_seg_b
  assert len(segment_ids) == len(input_ids)

  start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                  token_type_ids=torch.tensor([segment_ids]))

  answer_start = torch.argmax(start_scores)
  answer_end = torch.argmax(end_scores)

  answer = tokens[answer_start]

  for i in range(answer_start + 1, answer_end + 1):
      if tokens[i][0:2] == '##':
          answer += tokens[i][2:]
      else:
          answer += ' ' + tokens[i]

  print('Answer: "' + answer + '"')

Question:which novel by the author of armada is a movie by steven spielberg
Answer: "ready player one"
Question:who is the director of the film 2 states
Answer: "abhishek varman"
Question:which novel by the author of 2 states is a movie directed by mohit suri


IndexError: ignored